# 深度卷积对抗生成网络 ( 英语：Deep Convolution Generative Adversial Network，简称 ： DCGAN)

<br></br>

 * [在 Google Colab 运行](https://colab.research.google.com/github/dslab-deepflying/deepflying/blob/master/InnoCamp/GANS/dcgan.ipynb)
 * [浏览 GitHub 源码 ](https://github.com/JChrysanthemum/deepflying/blob/master/InnoCamp/GANS/dcgan.ipynb)

## 概述

<br>

生成对抗网络（英语：Generative Adversarial Network，简称GAN）是非监督式学习的一种方法，通过让两个神经网络相互博弈的方式进行学习。该方法由[伊恩·古德费洛等人](https://arxiv.org/abs/1406.2661)于2014年提出。

<br>

判别网络的输入则为真实样本或生成网络的输出，其目的是将生成网络的输出从真实样本中尽可能分辨出来。而生成网络则要尽可能地欺骗判别网络。两个网络相互对抗、不断调整参数，最终目的是使判别网络无法判断生成网络的输出结果是否真实。

<br>

  生成对抗网络常用于生成以假乱真的图片。Alec Radford等人于2015年又提出了一种结合 CNN（卷积神经网络） 与 GAN 结合的机器学习的模型结构，[深度卷积对抗网络](https://arxiv.org/abs/1511.06434)(英语：Deep Convolutional Generative Adversarial Networks，简称DCGAN)。 

##  环境搭建

###  所需包的导入

<br>

包（英文：package）是 python 中对一系列方法数据的封装，有面向对象编程（英文：Obect Oriented Coding，简称 OO）经验的同学来说并不陌生。对于没有OO基础的同学来说，包 可以简单的；理解为一种类似 “工具箱” 的概念。

<br>

我们本次实验使用的核心的包是[TensorFlow](https://www.tensorflow.org/?hl=zh_cn) ，谷歌公司的机器学习框架，也是当今世界上应用最广，发展最迅速的机器学习框架。[Keras](https://keras.io/zh/)是基于TensorFlow框架设计的高层抽象机器学习包。它使得我们能够略过数据操作中的张量操作以及计算图的编写运行等相对底层的操作，通过简单的 序贯模型 和 函数模型 快速搭建需要的机器学习模型。

<br>

其他包：<br>
1）matplotlib ， python用于图像表格绘制的包<br>
2）numpy ，python的数据运算工具包<br>
3）pandas，python用于数据文件操作工具包<br>


In [0]:
from __future__ import print_function, division


from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
%matplotlib inline

import sys,os
import numpy as np

import pandas as pd

**注意：
<br>由于Google Colab 不支持TensorFlow 2.0 ，本代码使用1.13版本
<br>最新的 TensorFlow2.0 版本 相较于此版本代码有以下改动**<br>
<br>1）keras包不必单独导入，可从TensorFlow直接导入。例如：<br>
```
    import keras                      // 旧方法

    import tensorflow.keras           // 方法一

    import tensorflow as tf           // 方法二
    from tf.keras import *          

```

<br>2）keras 旧版本繁杂包管理更新。例如：
```
    //旧版
    from keras.layers.advanced_activations import LeakyReLU
    from keras.layers.convolutional import UpSampling2D, Conv2D
    
    //新版
     from tensorflow.keras.layers import LeakyReLU
     from tensorflow.keras.layers import UpSampling2D, Conv2D
```


### 所需文件下载以及解压

[FashionMNIST](https://www.kaggle.com/zalando-research/fashionmnist)是德国Zalando公司的开源数据，分别有T恤、POLO衫、短裤、裙子、高跟鞋等多种服饰数据，每项数据为28X28的灰度图。

本次实验所需的数据我已经为大家进行了了分开整理，方便大家使用。


In [0]:
import os

if not os.path.exists('GAN/data'):
    os.makedirs('GAN/data')  
!wget https://github.com/JChrysanthemum/deepflying/raw/master/InnoCamp/GANS/data/data.zip -O GAN/data/data.zip
!unzip -o GAN/data/data.zip -d GAN/data/

if not os.path.exists('GAN/models'):
    os.makedirs('GAN/models')
    
if not os.path.exists('GAN/images'):
    os.makedirs('GAN/images')

## 代码编写

接下来，我们将一步一步组建深度卷积对抗生成网络模型

### 超参数定义

超参数是决定整个AI模型训练结果的 全局参数。在开头设置方便大家随时调整观察不同的结果。

In [0]:
IMG_ROWS = 28                           # 图片的行数，不用修改
IMG_COLS = 28                           # 图片的列数，不用修改
IMG_DATA_PATH = 'GAN/data/'             # 图片的路径，不用修改

#@markdown 参数修改
#@markdown ---
#@markdown <br>
# 训练数据的名称，可以自行修改

#@markdown 训练数据名（默认 tee ）：
data_name = 'tee'  #@param ['tee','ankle_boot', 'bag', 'coat', 'dress','pullover','sandal','shirt','sneaker','trouser']


"""
ankle_boot  ---------  高跟鞋

bag  ----------------  包

coat  ---------------  外套

dress  --------------  裙子

pullover  -----------  套头衫

sandal  -------------  凉鞋

shirt  --------------  衬衫

sneaker  ------------  运动鞋

tee  ----------------  T恤

trouser  ------------  裤子

"""

# 训练的批次数，在不考虑过拟合的情况下越大越好
#@markdown 训练批次 （默认 100 ）： 
EPOCHS = 3100 #@param {type:"slider", min:0, max:7000, step:100}

# 训练的批次大小，每次迭代需要的数据量，在资源足够下越大越好（越大越慢）
#@markdown 训练批次大小 （默认 2 ）：
BATCH_SIZE = 2 #@param [2,4,8,16,32]

# 保存图片的间隔数
#@markdown 图片保存和显示间隔 （默认 5 ）：
SAVE_INTERVAL = 5 #@param {type:"slider", min:0, max:2000, step:5}

#@markdown ---

### 网络定义

#### 定义构造函数 （初始化函数）

In [0]:
def __init__(self):
    # Input shape
    self.img_rows = IMG_ROWS
    self.img_cols = IMG_COLS
    self.channels = 1
    self.img_shape = (IMG_ROWS, IMG_COLS, self.channels)
    self.latent_dim = 100

    optimizer = Adam(0.0002, 0.5)

    # Build and compile the discriminator
    self.discriminator = self.build_discriminator()
    self.discriminator.compile(loss='binary_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])

    # Build the generator
    self.generator = self.build_generator()

    # The generator takes noise as input and generates imgs
    z = Input(shape=(self.latent_dim,))
    img = self.generator(z)

    # For the combined model we will only train the generator
    self.discriminator.trainable = False

    # The discriminator takes generated images as input and determines validity
    valid = self.discriminator(img)

    # The combined model  (stacked generator and discriminator)
    # Trains the generator to fool the discriminator
    self.combined = Model(z, valid)
    self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)



####  定义构造生成器

In [0]:
def build_generator(self):

    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    # model.summary()

    noise = Input(shape=(self.latent_dim,))
    img = model(noise)

    return Model(noise, img)



#### 定义构造判别器

In [0]:
def build_discriminator(self):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # model.summary()

    img = Input(shape=self.img_shape)
    validity = model(img)

    return Model(img, validity)



定义训练方法和参数

In [0]:
def train(self, epochs, batch_size=128, save_interval=50):

    # Load the dataset
    te = pd.read_csv(IMG_DATA_PATH + data_name + '.csv', header=None)
    X_train = []

    for i in range(1, len(te)):
        img = np.uint8(np.array(te.iloc[i]).reshape(28, 28))
        X_train.append(img)

    X_train = np.array(X_train)

    # Rescale -1 to 1
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Sample noise and generate a batch of new images
        noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Train the discriminator (real classified as ones and generated as zeros)
        d_loss_real = self.discriminator.train_on_batch(imgs, valid)
        d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        # Train the generator (wants discriminator to mistake images as real)
        g_loss = self.combined.train_on_batch(noise, valid)

        # Plot the progress
        sys.stdout.write(
            "\r%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))
        sys.stdout.flush()

        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            self.save_imgs(epoch)
    self.generator.save('GAN/models/dcgan_G_%s_%d.h5' % (data_name, epochs))
    self.discriminator.save('GAN/models/dcgan_D_%s_%d.h5' % (data_name, epochs))
    print('\n')




#### 定义保存和显示图片的函数

In [0]:

def save_imgs(self, epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, self.latent_dim))
    gen_imgs = self.generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig_name = "GAN/images/dcgan_%s_%d.png" % (data_name, epoch)
    if os.path.exists(sys.path[0] + "/" + fig_name):
        os.remove(sys.path[0] + "/" + fig_name)
    fig.savefig(fig_name)
    plt.show()
    plt.close()

#### 构造封装DCGAN类以便使用

In [0]:
class DCGAN():
  pass

DCGAN.__init__ = __init__
DCGAN.build_generator = build_generator
DCGAN.build_discriminator = build_discriminator
DCGAN.train = train
DCGAN.save_imgs = save_imgs

## 开始训练

In [0]:
if __name__ == "__main__":
  dcgan = DCGAN()
  dcgan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, save_interval=SAVE_INTERVAL)
